In [223]:
# Wrangling
import pandas as pd

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

### Loading the swiss data set
from pydataset import data

df = data('swiss')

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import tree

In [224]:
df.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [225]:
df.dtypes

Fertility           float64
Agriculture         float64
Examination           int64
Education             int64
Catholic            float64
Infant.Mortality    float64
dtype: object

In [226]:
df['is_catholic'] = df['Catholic'] > 70

In [227]:
df.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality,is_catholic
Courtelary,80.2,17.0,15,12,9.96,22.2,False
Delemont,83.1,45.1,6,9,84.84,22.2,True
Franches-Mnt,92.5,39.7,5,5,93.40,20.2,True
Moutier,85.8,36.5,12,7,33.77,20.3,False
Neuveville,76.9,43.5,17,15,5.16,20.6,False


In [228]:
df['is_catholic'] = df['is_catholic'].astype('category',copy=False)
df = df.drop(columns='Catholic')

In [229]:
df.dtypes

Fertility            float64
Agriculture          float64
Examination            int64
Education              int64
Infant.Mortality     float64
is_catholic         category
dtype: object

In [230]:
df.head()

,Fertility,Agriculture,Examination,Education,Infant.Mortality,is_catholic
Courtelary,80.2,17.0,15,12,22.2,False
Delemont,83.1,45.1,6,9,22.2,True
Franches-Mnt,92.5,39.7,5,5,20.2,True
Moutier,85.8,36.5,12,7,20.3,False
Neuveville,76.9,43.5,17,15,20.6,False


In [250]:
X = df.drop(columns='is_catholic')
y = df[['is_catholic']]
y = y.astype('int')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .40, random_state = 123, stratify=y)

X_train.head()

,Fertility,Agriculture,Examination,Education,Infant.Mortality
Broye,83.8,70.2,16,7,23.6
Franches-Mnt,92.5,39.7,5,5,20.2
Rive Droite,44.7,46.6,16,29,18.2
Porrentruy,76.1,35.3,9,7,26.6
La Chauxdfnd,65.7,7.7,29,11,20.5


# Decision Tree Classifier

In [251]:
clf = DecisionTreeClassifier(max_depth=2, random_state=123)

In [252]:
clf.fit(X_train.drop(columns=['Agriculture', 'Examination', 'Infant.Mortality']), y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

In [253]:
y_pred = clf.predict(X_train.drop(columns=['Agriculture', 'Examination', 'Infant.Mortality']))
y_pred_proba = clf.predict_proba(X_train.drop(columns=['Agriculture', 'Examination', 'Infant.Mortality']))

In [254]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train.drop(columns=['Agriculture', 'Examination', 'Infant.Mortality']), y_train)))

Accuracy of Decision Tree classifier on training set: 0.93


In [255]:
confusion_matrix(y_train, y_pred)

array([[17,  1],
       [ 1,  9]])

In [256]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94        18
           1       0.90      0.90      0.90        10

   micro avg       0.93      0.93      0.93        28
   macro avg       0.92      0.92      0.92        28
weighted avg       0.93      0.93      0.93        28



# Logistic Regression Model

In [257]:
logit = LogisticRegression(C=1, class_weight={1:2}, random_state = 123, solver='saga')

In [258]:
logit.fit(X_train.drop(columns=['Fertility', 'Education', 'Infant.Mortality']), y_train)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1, class_weight={1: 2}, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=123, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

In [259]:
print('Coefficient: \n', logit.coef_)
print('Intercept: \n', logit.intercept_)

Coefficient: 
 [[ 0.05562038 -0.22402316]]
Intercept: 
 [0.01639362]


In [260]:
y_pred = logit.predict(X_train.drop(columns=['Fertility', 'Education', 'Infant.Mortality']))

y_pred_proba = logit.predict_proba(X_train.drop(columns=['Fertility', 'Education', 'Infant.Mortality']))

In [261]:
print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(logit.score(X_train.drop(columns=['Fertility', 'Education', 'Infant.Mortality']), y_train)))

Accuracy of Logistic Regression classifier on training set: 0.79


In [262]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.78      0.82        18
           1       0.67      0.80      0.73        10

   micro avg       0.79      0.79      0.79        28
   macro avg       0.77      0.79      0.78        28
weighted avg       0.80      0.79      0.79        28



# K Nearest Neighbors model

In [263]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

In [264]:
knn.fit(X_train.drop(columns=['Agriculture', 'Examination', 'Education']), y_train)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [265]:
y_pred = knn.predict(X_train.drop(columns=['Agriculture', 'Examination', 'Education']))

y_pred_proba = knn.predict_proba(X_train.drop(columns=['Agriculture', 'Examination', 'Education']))

In [266]:
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(X_train.drop(columns=['Agriculture', 'Examination', 'Education']), y_train)))

Accuracy of KNN classifier on training set: 0.89


In [267]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.94      0.92        18
           1       0.89      0.80      0.84        10

   micro avg       0.89      0.89      0.89        28
   macro avg       0.89      0.87      0.88        28
weighted avg       0.89      0.89      0.89        28



# Best Model: Decision Tree Classifier

In [268]:
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(logit.score(X_test.drop(columns=['Fertility', 'Education', 'Infant.Mortality']), y_test)))

Accuracy of Decision Tree classifier on test set: 0.95
